In [227]:
import time
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm

import chromedriver_binary
from bs4 import BeautifulSoup
from selenium import webdriver

month_dct = {'Jan' : 1, 'Feb' : 2, 'Mar' : 3, 
             'Apr' : 4, 'May' : 5, 'Jun' : 6,
             'Jul' : 7, 'Aug' : 8, 'Sep' : 9, 
             'Oct' : 10,'Nov' : 11, 'Dec' : 12}

driver = webdriver.Chrome('/Users/ashishzangra/opt/anaconda3/lib/python3.9/site-packages/chromedriver_binary/chromedriver')

driver.get('https://www.kaggle.com/')

/var/folders/rq/lwddjvtn5n9gjw05hwrg3hp80000gn/T/ipykernel_1082/2700938162.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/ashishzangra/opt/anaconda3/lib/python3.9/site-packages/chromedriver_binary/chromedriver')


### 1. Scraping Data from Kaggle

In [231]:
data = []

for page in tqdm(range(1,501)):                     # Going through each page

                                                    # Creating Page Link
    link = 'https://www.kaggle.com/datasets?page=' + str(page)
    driver.get(link)                                # Sending Request to that link
     
    time.sleep(2)                                   # Adding delay of 2 seconds so that the whole page is opened
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')            # Converitng page to soup
    sp = soup.find('ul', class_ = 'km-list km-list--three-line')       # Shortlisting dataset


    for dataset in sp.find_all('li'):               # Going through each dataset
 

        try:                                        # Extracting Dataset Link
            dataset_link = 'https://www.kaggle.com' + dataset.find('a').get('href')
        except:
            dataset_link = np.nan

        try:                                        # Image Link
            img_link     = dataset.find('div',class_ = 'sc-ieieiu hkWzeY').get('style')
            img_link     = 'https' + img_link.split('https')[1].split('?')[0]
        except:
            img_link     = np.nan

        try:                                        # Extracting title of the dataset
            title        = dataset.find('div',class_ = 'sc-iCoGMd sc-fKgJPI sc-ckTSus cHRTxu cxbltl bVSQFd').text.strip()
        except:
            title        = np.nan

        try:                                        # Extracting name of the author
            author_name  = dataset.find_all('span',class_ = 'sc-jJMGnK sc-hiKfDv sc-FRrlG lkfHZi dRsyqq bFavVq')[0].find('a').text.strip()
        except:
            author_name  = np.nan

        try:                                        # Extracting the author id
            author_id    = 'https://www.kaggle.com' + dataset.find_all('span',class_ = 'sc-jJMGnK sc-hiKfDv sc-FRrlG lkfHZi dRsyqq bFavVq')[0].find('a').get('href')
        except:
            author_id    = np.nan


        try:                                        # Extracting Date-time
            date_time    = dataset.find_all('span',class_ = 'sc-jJMGnK sc-hiKfDv sc-FRrlG lkfHZi dRsyqq bFavVq')[0].find('span').get('title')[:-31].strip()

            temp_date    = date_time.split(" ")[1:-1]
            temp_date[0] = str(month_dct[temp_date[0]])

            date         = '/'.join(temp_date)
            day          = date_time.split(' ')[0]
            time_         = date_time.split(' ')[-1]

        except:
            date_time    = np.nan
            date         = np.nan
            day          = np.nan
            time_         = np.nan

        try:                                         # Extracting Usability
            usability    = float(dataset.find_all('span',class_ = 'sc-jJMGnK sc-hiKfDv sc-FRrlG lkfHZi dRsyqq bFavVq')[1].find('span').text.strip())
        except:
            usability    = np.nan

        try:                                        # Basic Data Information
            data_info    = dataset.find_all('span',class_ = 'sc-jJMGnK sc-hiKfDv sc-FRrlG lkfHZi dRsyqq bFavVq')[1].text.split('·')[1].strip()
            no_of_files  = int(data_info.split(' ')[0])
            file_type    = data_info.split(' ')[-1][1:-1]
        except:
            data_info    = np.nan
            no_of_files  = np.nan

        try:                                        # Extracting Size
            size         = dataset.find_all('span',class_ = 'sc-jJMGnK sc-hiKfDv sc-FRrlG lkfHZi dRsyqq bFavVq')[1].text.split('·')[2].strip()
        except:
            size         = np.nan

        try:                                        # Extracting Upvotes
            upvotes      = int(dataset.find('span', mode = 'default').text.strip())
        except:
            upvotes      = np.nan


        try:                                        # Extracting Medal
            medals       = dataset.find('span', class_ = 'sc-jJMGnK sc-hiKfDv lkfHZi dRsyqq').text.strip()
        except:
            medals       = np.nan

        # Adding All data in a list
        data.append([title, author_name, author_id, size, file_type, no_of_files, upvotes, medals, usability,
                     date, day,time_, dataset_link, img_link])

100%|█████████████████████████████████████████| 500/500 [25:19<00:00,  3.04s/it]


### 2. Saving Data in a DataFrame

In [268]:
df = pd.DataFrame(data, columns = ['title', 'author_name', 'author_id','size','file_type','no_of_files',
                             'upvotes','medal','usability','date','day','time','dataset_link','img_link'])

df.head()

,title,author_name,author_id,size,file_type,no_of_files,upvotes,medal,usability,date,day,time,dataset_link,img_link
0,Netflix TV Shows and Movies,Victor Soeiro,https://www.kaggle.com/victorsoeiro,2 MB,CSV,2,180,Bronze,10.0,5/15/2022,Sun,05:31:23,https://www.kaggle.com/datasets/victorsoeiro/n...,NaN
1,Supermarket store branches sales analysis,Suraj Jha,https://www.kaggle.com/surajjha101,10 kB,CSV,1,154,Gold,10.0,4/29/2022,Fri,16:40:16,https://www.kaggle.com/datasets/surajjha101/st...,NaN
2,Student Performance Dataset,Dev Ansodariya,https://www.kaggle.com/devansodariya,7 kB,CSV,1,108,Bronze,9.7,5/26/2022,Thu,19:25:09,https://www.kaggle.com/datasets/devansodariya/...,NaN
3,Top Hits Spotify from 2000-2019,Mark Koverha,https://www.kaggle.com/paradisejoy,96 kB,CSV,1,120,Silver,10.0,5/31/2022,Tue,12:50:57,https://www.kaggle.com/datasets/paradisejoy/to...,NaN
4,Amex Competition Data in Parquet Format,Sanskar Hasija,https://www.kaggle.com/odins0n,9 GB,other,2,83,Gold,10.0,5/26/2022,Thu,04:50:19,https://www.kaggle.com/datasets/odins0n/amex-p...,NaN


### 3. Checking Null Values

In [269]:
df.isnull().sum()

title              0
author_name      424
author_id        424
size             680
file_type          0
no_of_files       26
upvotes           24
medal           4544
usability         27
date              24
day               24
time              24
dataset_link      24
img_link        9656
dtype: int64

### 4. Saving Raw Dataset

In [270]:
df.to_csv('dataset_raw.csv',index = False)

### 5. Remove Unnecesary Columns

In [271]:
del df['img_link']

df['medal'] = df['medal'].fillna('No Medal')

In [272]:
df.isnull().sum()

title             0
author_name     424
author_id       424
size            680
file_type         0
no_of_files      26
upvotes          24
medal             0
usability        27
date             24
day              24
time             24
dataset_link     24
dtype: int64

In [273]:
df = df.dropna()

df.head()

,title,author_name,author_id,size,file_type,no_of_files,upvotes,medal,usability,date,day,time,dataset_link
0,Netflix TV Shows and Movies,Victor Soeiro,https://www.kaggle.com/victorsoeiro,2 MB,CSV,2,180,Bronze,10.0,5/15/2022,Sun,05:31:23,https://www.kaggle.com/datasets/victorsoeiro/n...
1,Supermarket store branches sales analysis,Suraj Jha,https://www.kaggle.com/surajjha101,10 kB,CSV,1,154,Gold,10.0,4/29/2022,Fri,16:40:16,https://www.kaggle.com/datasets/surajjha101/st...
2,Student Performance Dataset,Dev Ansodariya,https://www.kaggle.com/devansodariya,7 kB,CSV,1,108,Bronze,9.7,5/26/2022,Thu,19:25:09,https://www.kaggle.com/datasets/devansodariya/...
3,Top Hits Spotify from 2000-2019,Mark Koverha,https://www.kaggle.com/paradisejoy,96 kB,CSV,1,120,Silver,10.0,5/31/2022,Tue,12:50:57,https://www.kaggle.com/datasets/paradisejoy/to...
4,Amex Competition Data in Parquet Format,Sanskar Hasija,https://www.kaggle.com/odins0n,9 GB,other,2,83,Gold,10.0,5/26/2022,Thu,04:50:19,https://www.kaggle.com/datasets/odins0n/amex-p...


In [274]:
df.isnull().sum()

title           0
author_name     0
author_id       0
size            0
file_type       0
no_of_files     0
upvotes         0
medal           0
usability       0
date            0
day             0
time            0
dataset_link    0
dtype: int64

### 6. Saving the preprocessed Dataset

In [275]:
df.to_csv('datasets.csv',index = False)